<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D9%85%D8%A7%D9%83%D8%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

# استخدام الدالة
data = calculate_macd(data)
ذي

In [ ]:
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


In [ ]:
data = get_and_clean_gold_prices()
if data is not None:
    data = calculate_bollinger_bands(data)  # إضافة هذه السطر لحساب بولينجر باندز
    build_strategy(data)


In [ ]:
def calculate_stochastic(data, k_window=14, d_window=3):
    """حساب Stochastic Oscillator"""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()
    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()
    return data


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)